## Agentic Workflow with LangGraph & SingleStore Database 

Apart from the pre added custom knowledge sources, added web search tool for unknown questions

#### Install required packages

In [22]:
!pip install langgraph langchain langchain-community singlestoredb openai tiktoken requests beautifulsoup4 tavily-python --quiet

#### Import libraries

In [23]:
import os
import requests
from bs4 import BeautifulSoup
from typing import TypedDict, List, Optional
from langchain_community.vectorstores import SingleStoreDB
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document
from langchain_openai import ChatOpenAI
from langgraph.graph import END, StateGraph
from tavily import TavilyClient

#### Set credentials

In [24]:
os.environ["OPENAI_API_KEY"] = "Add Your OpenAI API Key"
os.environ["SINGLESTOREDB_URL"] = "admin:password@SingleStore-host-url:3306/dbname"
os.environ["TAVILY_API_KEY"] = "Add Your Tavily API Key"

#### URL Content Loader

In [25]:
def load_url_content(url):
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}  # Avoid 403 errors
        response = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Remove irrelevant elements
        for element in soup(["script", "style", "header", "footer", "nav", "aside"]):
            element.decompose()
            
        return ' '.join(soup.stripped_strings)
    
    except Exception as e:
        print(f"Error loading {url}: {str(e)}")
        return ""

#### Load content from URLs

In [26]:
urls = [
    "https://en.wikipedia.org/wiki/Large_language_model",
    "https://en.wikipedia.org/wiki/Retrieval-augmented_generation"
]
print("⏳ Loading content from URLs...")
documents = [load_url_content(url) for url in urls]
combined_content = "\n\n".join(docs for docs in documents if docs)

⏳ Loading content from URLs...


#### Split documents

In [27]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
print("✂️ Splitting documents into chunks...")
docs = text_splitter.create_documents([combined_content])

✂️ Splitting documents into chunks...


#### Initialize SingleStore vector store

In [28]:
embeddings = OpenAIEmbeddings()
print("💾 Storing documents in SingleStore...")
vectorstore = SingleStoreDB.from_documents(
    docs,
    embeddings,
    table_name="agent_knowledge_base",
    distance_strategy="DOT_PRODUCT"
)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

💾 Storing documents in SingleStore...


#### Web Search Tool

In [29]:
tavily = TavilyClient()

def web_search(query: str, max_results: int = 3) -> str:
    """Perform web search and return summarized results"""
    print(f"🌐 Searching web for: {query}")
    results = tavily.search(query=query, max_results=max_results)
    return "\n\n".join(
        f"🔗 Source {i+1}: [{r['title']}]({r['url']})\n{r['content']}" 
        for i, r in enumerate(results["results"])
    )

#### Build the RAG chain

In [30]:
model = ChatOpenAI(model="gpt-3.5-turbo")
output_parser = StrOutputParser()

template = """Answer the question using ANY of these sources:
{context}

Question: {question}
If information conflicts, prioritize local knowledge base.
If you don't know, say you don't know - don't make up answers."""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {"context": RunnablePassthrough(), "question": RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)

#### Define the State Structure

In [31]:
class AgentState(TypedDict):
    question: str
    context: List[Document]
    web_context: Optional[str]
    answer: str
    next_step: str  # Track next action

#### Create Graph Nodes

In [32]:
def retrieve_node(state: AgentState):
    """Retrieve documents from SingleStore"""
    print(f"🔍 Retrieving documents for: {state['question']}")
    context = retriever.invoke(state["question"])
    return {"context": context}

def web_search_node(state: AgentState):
    """Perform web search for unknown questions"""
    web_results = web_search(state["question"])
    return {"web_context": web_results, "next_step": "generate"}

def generate_node(state: AgentState):
    """Generate final answer"""
    # Combine context sources
    local_context = "\n\n".join(
        f"📄 Document {i+1}:\n{doc.page_content}" 
        for i, doc in enumerate(state["context"])
    )
    
    if state.get("web_context"):
        full_context = f"LOCAL KNOWLEDGE BASE:\n{local_context}\n\nWEB RESULTS:\n{state['web_context']}"
    else:
        full_context = f"LOCAL KNOWLEDGE BASE:\n{local_context}\n\nWEB RESULTS: None available"
    
    # Generate answer
    print("💡 Generating answer...")
    result = chain.invoke({
        "question": state["question"],
        "context": full_context
    })
    return {"answer": result}

def decision_node(state: AgentState):
    """Decide whether to use web search"""
    # Simple relevance check
    if not state["context"]:
        return {"next_step": "web_search"}
    
    # Check if context contains answer
    context_text = " ".join(doc.page_content for doc in state["context"])
    prompt = f"""Does this context contain information to answer the question?
    Question: {state['question']}
    Context: {context_text[:1000]}...
    Answer ONLY 'yes' or 'no':"""
    
    response = ChatOpenAI(model="gpt-3.5-turbo", temperature=0).invoke(prompt).content
    return {"next_step": "generate" if "yes" in response.lower() else "web_search"}

#### Build the Agent Workflow

In [33]:
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("retrieve", retrieve_node)
workflow.add_node("decision", decision_node)
workflow.add_node("web_search", web_search_node)
workflow.add_node("generate", generate_node)

# Set connections
workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "decision")
workflow.add_conditional_edges(
    "decision",
    lambda state: state["next_step"],
    {
        "web_search": "web_search",
        "generate": "generate"
    }
)
workflow.add_edge("web_search", "generate")
workflow.add_edge("generate", END)

# Compile the agent
print("🤖 Building agent...")
agent = workflow.compile()

🤖 Building agent...


#### Run the agent

In [34]:
def ask_question(question: str):
    """Run the agent with a question"""
    print(f"\n❓ Question: {question}")
    state = {
        "question": question,
        "context": [],
        "web_context": None,
        "answer": "",
        "next_step": "retrieve"
    }
    result = agent.invoke(state)
    print(f"\n✅ Answer: {result['answer']}")

# Test with different questions
ask_question("What is a large language model?")  # Should use local knowledge
ask_question("What are the latest AI developments announced this month?")


❓ Question: What is a large language model?
🔍 Retrieving documents for: What is a large language model?
💡 Generating answer...

✅ Answer: A large language model is a model that has the capacity to process and understand language on a large scale. It can range from statistical language models to neural network-based models, with the ability to effectively ingest and analyze large datasets for various language processing tasks.

❓ Question: What are the latest AI developments announced this month?
🔍 Retrieving documents for: What are the latest AI developments announced this month?
🌐 Searching web for: What are the latest AI developments announced this month?
💡 Generating answer...

✅ Answer: The latest AI developments announced this month include Google's announcement of Imagen 4, the latest version of its AI text-to-image generator, which offers improved text generation capabilities and the ability to export images in various formats. This was one of the significant announcements at G

In [36]:
ask_question("What is NVIDIA's latest contribution to large language models (LLMs)?")


❓ Question: What is NVIDIA's latest contribution to large language models (LLMs)?
🔍 Retrieving documents for: What is NVIDIA's latest contribution to large language models (LLMs)?
🌐 Searching web for: What is NVIDIA's latest contribution to large language models (LLMs)?
💡 Generating answer...

✅ Answer: NVIDIA's latest contribution to large language models is the introduction of the NVLM 1.0 family of frontier-class multimodal LLMs. These models achieve state-of-the-art results on vision-language tasks and show improved text-only performance over their LLM backbone after multimodal training.


In [40]:
ask_question("What is Microsoft's latest AI news in June 2025?")


❓ Question: What is Microsoft's latest AI news in June 2025?
🔍 Retrieving documents for: What is Microsoft's latest AI news in June 2025?
🌐 Searching web for: What is Microsoft's latest AI news in June 2025?
💡 Generating answer...

✅ Answer: Microsoft's latest AI news in June 2025 includes the announcement of Microsoft 365 Copilot Tuning, multi-agent orchestration, and more at Microsoft Build 2025. This new capability in Microsoft Copilot Studio allows organizations to tune AI models using their own company data, workflows, and processes without code.
